# Table of Contents
* [Exploring fenaroli parsing](#Exploring-fenaroli-parsing)
	* [Loading plaintext generated by slate (on AWS instance)](#Loading-plaintext-generated-by-slate-%28on-AWS-instance%29)
	* [RE for flavor compounds](#RE-for-flavor-compounds)
	* [RE for ingredients](#RE-for-ingredients)


In [689]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, deque
%load_ext autoreload
%autoreload 2

In [3]:
import cPickle as pickle
import slate 
import re

In [6]:
import matplotlib.pylab as plt
import matplotlib as mpl
plt.rcParams['figure.figsize'] = (12.0, 6.0)
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default

# Exploring fenaroli parsing

## Loading plaintext generated by slate (on AWS instance)

In [296]:
with open('../data/doc_joined.pkl', 'r') as f:
    joined = pickle.load(f)

In [295]:
cleaned_joined = joined.replace('.', ' ')

In [542]:
# cleaned_joined[:int(len(cleaned_joined)*0.2)]

In [688]:
raw_shard = cleaned_joined[int(len(cleaned_joined)*0.6):int(len(cleaned_joined)*0.7)]

In [738]:
raw_shard

In [375]:
with open ('joined_inspect', 'w') as f:
    f.write(cleaned_joined)

entries I'm interested in start at split doc 17

## RE for flavor compounds and ingredients

This is the ultimate regex pattern. It finds the natural occurence, and searches below for the next flavor compound
'Natural occurrence: (.+)\n{2,5}[\x0c]?(.*[A-Z]{2,20}.*)\n+'

In [ ]:
re.

In [ ]:
(?<=.{%d})' % (offset)

In [649]:
# sq = re.compile('Natural occurrence:(.+)\n\n[\x0c]?(.*?[A-Z]{2,20}.*?)\n\n')
sq = re.compile('Natural occurrence:(.+)\n\n[\x0c]?(.*?[A-Z]{2,20}.*?)\n\n')

raw_ingredients = re.findall(sq, cleaned_joined)
ingredient_series = pd.Series([i[1] for i in raw_ingredients])

In [683]:
raw_ingredients[:10]

## Here I have the material I need to build the ingredient:compound mapping

In [794]:
def flav_ingr_mapper(raw_text):
    
    def check_for_header(new_comp, blobs, i, comp_pattern):
        for j in range(i,i+50):
            try:
                if new_comp[0] == re.findall(comp_pattern, blobs[j]):
                    return True
            except IndexError:
                pass
        return False
    
    flavor_dict = defaultdict(list)
    ignore_terms = ['EINECS No :']
    re.MULTILINE
    ingr_pattern = re.compile('Natural occurrence:(.+)')
    comp_pattern = re.compile('(.*[A-Z]{6,20}.*)')
    
    blobs = raw_text.split('\n\n')
    comp_q = deque([], maxlen=3)
    for i in range(len(blobs)):
        text = blobs[i].replace('\n', '')
        ing = re.findall(ingr_pattern, text)
        comp = re.findall(comp_pattern, text)
        if comp and comp[0] not in ignore_terms:
            if not check_for_header(comp, blobs, i, comp_pattern):
                comp_q.append(comp[0])
        if ing:
            try:
                flavor_dict[comp_q.pop()] = ing[0]
            except IndexError:
                pass
    return flavor_dict

In [795]:
flav_comp_dict = flav_ingr_mapper(raw_shard)

In [783]:
flavor_dict = defaultdict(list)
for i in range(1, len(raw_ingredients)):
    flavor_dict[raw_ingredients[i-1][1].strip()] = raw_ingredients[i][0].strip()

This is the compound - ingredient mapping I've been striving for

In [671]:
pd.Series(flavor_dict.values()).value_counts()
remove_terms = ['Not reported found in nature','n/a','Reported found in nature', 'Reported not found in nature']

In [677]:
fd = {k:v for k,v in flavor_dict.iteritems() if v not in remove_terms}

In [797]:
len(flav_comp_dict.keys())